___
# Simulação de Monte Carlo Aplicada a GERG-2008 para a fase vapor

In [1]:
# Essencials modules
from GERG2008rho import GERG2008rho # this function is the GERG-2008 to vapor fase
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd
import findK
from tqdm import tqdm
import time


# matplotlib notebook
#%matplotlib ipympl
%matplotlib qt

# figure options
plt.rcParams['font.family'] = ['serif']
plt.rcParams['font.serif'] = ['Times New Roman']
plt.rcParams['figure.autolayout'] = True
plt.rcParams['text.usetex'] = True

SMALL_SIZE = 14
MEDIUM_SIZE = 18
BIGGER_SIZE = 22

plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=BIGGER_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=SMALL_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=MEDIUM_SIZE)   # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title

___
## Variáveis de entrada

* c: Lista contendo a proporção molar de cada componente da mistura. A soma das proporções molares deve ser igual a um, ou seja: 

    $\LARGE\sum_ {i=0} ^ n c_i = 1 $
        
* T: Temperatura em Kelvin (K)

* p: Pressão absoluta em quilopascal (kPa)


In [2]:
# Molar composition
c = np.zeros((2,21))

# Multicomponents mixture from Baladão's dissertation.
# Mixture N1 → c0
c[0][0 ] = 0.34242        # Methane
c[0][1 ] = 0.00000        # Nitrogen
c[0][2 ] = 0.00000        # CO2
c[0][3 ] = 0.31372        # Ethane
c[0][4 ] = 0.34386        # Propane
c[0][5 ] = 0.00000        # Isobutane
c[0][6 ] = 0.00000        # Butane
c[0][7 ] = 0.00000        # Isopentane
c[0][8 ] = 0.00000        # Pentane
c[0][9 ] = 0.00000        # Hexane
c[0][10] = 0.00000        # Heptane
c[0][11] = 0.00000        # Octane
c[0][12] = 0.00000        # Nonane
c[0][13] = 0.00000        # Decane
c[0][14] = 0.00000        # Hydrogen
c[0][15] = 0.00000        # Oxygen
c[0][16] = 0.00000        # CO
c[0][17] = 0.00000        # Water
c[0][18] = 0.00000        # H2S
c[0][19] = 0.00000        # Helium
c[0][20] = 0.00000        # Argon

# Mixture N2 → c1
c[1][0 ] = 0.85260        # Methane
c[1][1 ] = 0.04840        # Nitrogen
c[1][2 ] = 0.00000        # CO2
c[1][3 ] = 0.04830        # Ethane
c[1][4 ] = 0.05070        # Propane
c[1][5 ] = 0.00000        # Isobutane
c[1][6 ] = 0.00000        # Butane
c[1][7 ] = 0.00000        # Isopentane
c[1][8 ] = 0.00000        # Pentane
c[1][9 ] = 0.00000        # Hexane
c[1][10] = 0.00000        # Heptane
c[1][11] = 0.00000        # Octane
c[1][12] = 0.00000        # Nonane
c[1][13] = 0.00000        # Decane
c[1][14] = 0.00000        # Hydrogen
c[1][15] = 0.00000        # Oxygen
c[1][16] = 0.00000        # CO
c[1][17] = 0.00000        # Water
c[1][18] = 0.00000        # H2S
c[1][19] = 0.00000        # Helium
c[1][20] = 0.00000        # Argon

___
## Range de validade da GERG-2008

A GERG-2008 possui um range de validade para temperaturas entre 60 e 700 K e pressão máxima de 70 MPa.
___
## Método da bisseção

Vou utilizar o método da bisseção para obtenção de intervalo de temperaturas no qual as misturas aprensentam a primeira fase líquida.
<img src="bisection.png" width="400" style = "display: block; margin-left: auto; margin-right: auto">

## Monte Carlo para a condição de temperatura de 150°C e pressões de 10, 20, 30, 40, 50, 60 e 70 MPa

### Cálculo da massa específica na condição padrão de medição 20°C e 101,325 kPa

In [3]:
# Standard condition
T_standard = 273.15+20  # Temperature in Kelvin
P_standard = 101.325    # Pressure in kPa

rho_standard = []

# Evaluanting for each mixture
for i in range(0,len(c)):
    rho_standard.append(GERG2008rho(c[i],P_standard,T_standard))


In [4]:
rho_standard

[0.04189683388202396, 0.04166929098293479]

# Condição de Distribuição Normal de 150°C e fixo 10 MPa. Condição 00

In [5]:
T0 = 100+273.15 # Temperature in Kelvin
P0 = 10000    # Pressure in kPa
# This function evaluating a normal samples of bounds of P and T values
sigma = 0.01*T0
ns = 100
mu = T0

samples_n150_10 = (sorted(np.random.normal(mu,sigma,ns).tolist()))

In [6]:
rho_n150_10 = [[] for _ in range(len(c))]

# Evaluanting for each mixture
for i in range(0,len(c)):
    for j, tempo in zip(range(len(samples_n150_10)), tqdm(range(len(samples_n150_10)))):
        rho_n150_10[i].append(GERG2008rho(c[i],P0,samples_n150_10[j]))

 99%|███████████████████████████████████████████████████████████████████████████████████████████████████████████▉ | 99/100 [24:26<00:14, 14.81s/it]


Cálculo da incerteza para a condição 00

In [7]:
# Evaluating r
r_00 = []
for i in range(len(rho_standard)):
    r_00.append( rho_n150_10[i]/rho_standard[i])

# Evaluating u_r
ur_00 = []
for i in range(0,len(c)):
    ur_00.append(np.std(r_00[i]))

# Evaluating the uncertaninty portion with 95% confidence
u_00 = []
for i in range(0,len(c)):
    u_00.append(((1.96*np.std(r_00[i]))/np.mean(r_00[i])))

In [8]:
print('Incerteza relativa de r00 para c0 =', round(u_00[0]*100,2),'%')
print('Incerteza relativa de r00 para c1 =', round(u_00[1]*100,2),'%')

Incerteza relativa de r00 para c0 = 6.27 %
Incerteza relativa de r00 para c1 = 2.72 %


### Condição de Distribuição Normal de 150°C e fixo 20 MPa. Condição 01

In [9]:
T0 = 100+273.15 # Temperature in Kelvin
P1 = 20000    # Pressure in kPa
# This function evaluating a normal samples of bounds of P and T values
sigma = 0.01*T0
ns = 100
mu = T0

samples_n150_20 = (sorted(np.random.normal(mu,sigma,ns).tolist()))

In [10]:
rho_n150_20 = [[] for _ in range(len(c))]

# Evaluanting for each mixture
for i in range(0,len(c)):
    for j, tempo in zip(range(len(samples_n150_20)), tqdm(range(len(samples_n150_20)))):
        rho_n150_20[i].append(GERG2008rho(c[i],P1,samples_n150_20[j]))

 99%|███████████████████████████████████████████████████████████████████████████████████████████████████████████▉ | 99/100 [23:41<00:14, 14.36s/it]


In [11]:
# Evaluating r
r_01 = []
for i in range(len(rho_standard)):
    r_01.append( rho_n150_20[i]/rho_standard[i])

# Evaluating u_r
ur_01 = []
for i in range(0,len(c)):
    ur_01.append(np.std(r_01[i]))

# Evaluating the uncertaninty portion with 95% confidence
u_01 = []
for i in range(0,len(c)):
    u_01.append(((1.96*np.std(r_01[i]))/np.mean(r_01[i])))

In [12]:
print('Incerteza relativa de r01 para c0 =', round(u_01[0]*100,2),'%')
print('Incerteza relativa de r01 para c1 =', round(u_01[1]*100,2),'%')

Incerteza relativa de r01 para c0 = 3.98 %
Incerteza relativa de r01 para c1 = 3.04 %


### Condição de Distribuição Normal de 150°C e fixo 30 MPa. Condição 02

In [13]:
T0 = 100+273.15 # Temperature in Kelvin
P2 = 30000    # Pressure in kPa
# This function evaluating a normal samples of bounds of P and T values
sigma = 0.01*T0
ns = 100
mu = T0

samples_n150_30 = (sorted(np.random.normal(mu,sigma,ns).tolist()))

In [14]:
rho_n150_30 = [[] for _ in range(len(c))]

# Evaluanting for each mixture
for i in range(0,len(c)):
    for j, tempo in zip(range(len(samples_n150_30)), tqdm(range(len(samples_n150_30)))):
        rho_n150_30[i].append(GERG2008rho(c[i],P2,samples_n150_30[j]))

 99%|███████████████████████████████████████████████████████████████████████████████████████████████████████████▉ | 99/100 [23:42<00:14, 14.36s/it]


In [15]:
# Evaluating r
r_02 = []
for i in range(len(rho_standard)):
    r_02.append( rho_n150_30[i]/rho_standard[i])

# Evaluating u_r
ur_02 = []
for i in range(0,len(c)):
    ur_02.append(np.std(r_02[i]))

# Evaluating the uncertaninty portion with 95% confidence
u_02 = []
for i in range(0,len(c)):
    u_02.append(((1.96*np.std(r_02[i]))/np.mean(r_02[i])))

In [16]:
print('Incerteza relativa de r02 para c0 =', round(u_02[0]*100,2),'%')
print('Incerteza relativa de r02 para c1 =', round(u_02[1]*100,2),'%')

Incerteza relativa de r02 para c0 = 2.48 %
Incerteza relativa de r02 para c1 = 2.68 %


### Condição de Distribuição Normal de 4°C e fixo 115 MPa. Condição 03

In [17]:
T0 = 100+273.15 # Temperature in Kelvin
P3 = 40000    # Pressure in kPa
# This function evaluating a normal samples of bounds of P and T values
sigma = 0.01*T0
ns = 100
mu = T0

samples_n150_40 = (sorted(np.random.normal(mu,sigma,ns).tolist()))

In [18]:
rho_n150_40 = [[] for _ in range(len(c))]

# Evaluanting for each mixture
for i in range(0,len(c)):
    for j, tempo in zip(range(len(samples_n150_40)), tqdm(range(len(samples_n150_40)))):
        rho_n150_40[i].append(GERG2008rho(c[i],P3,samples_n150_40[j]))

 99%|███████████████████████████████████████████████████████████████████████████████████████████████████████████▉ | 99/100 [23:37<00:14, 14.32s/it]


In [19]:
# Evaluating r
r_03 = []
for i in range(len(rho_standard)):
    r_03.append( rho_n150_40[i]/rho_standard[i])

# Evaluating u_r
ur_03 = []
for i in range(0,len(c)):
    ur_03.append(np.std(r_03[i]))

# Evaluating the uncertaninty portion with 95% confidence
u_03 = []
for i in range(0,len(c)):
    u_03.append(((1.96*np.std(r_03[i]))/np.mean(r_03[i])))

In [20]:
print('Incerteza relativa de r03 para c0 =', round(u_03[0]*100,2),'%')
print('Incerteza relativa de r03 para c1 =', round(u_03[1]*100,2),'%')

Incerteza relativa de r03 para c0 = 2.04 %
Incerteza relativa de r03 para c1 = 2.47 %


### Condição de Distribuição Normal de 150°C e fixo 50 MPa. Condição 04

In [21]:
T0 = 100+273.15 # Temperature in Kelvin
P4 = 50000    # Pressure in kPa
# This function evaluating a normal samples of bounds of P and T values
sigma = 0.01*T0 
ns = 100
mu = T0

samples_n150_50 = (sorted(np.random.normal(mu,sigma,ns).tolist()))

In [34]:
rho_n150_50 = [[] for _ in range(len(c))]

# Evaluanting for each mixture
for i in range(0,len(c)):
    for j, tempo in zip(range(len(samples_n150_50)), tqdm(range(len(samples_n150_50)))):
        rho_n150_50[i].append(GERG2008rho(c[i],P4,samples_n150_50[j]))

 99%|███████████████████████████████████████████████████████████████████████████████████████████████████████████▉ | 99/100 [24:05<00:14, 14.60s/it]


In [35]:
# Evaluating r
r_04 = []
for i in range(len(rho_standard)):
    r_04.append( rho_n150_50[i]/rho_standard[i])

# Evaluating u_r
ur_04 = []
for i in range(0,len(c)):
    ur_04.append(np.std(r_04[i]))

# Evaluating the uncertaninty portion with 95% confidence
u_04 = []
for i in range(0,len(c)):
    u_04.append(((1.96*np.std(r_04[i]))/np.mean(r_04[i])))

In [36]:
print('Incerteza relativa de r04 para c0 =', round(u_04[0]*100,2),'%')
print('Incerteza relativa de r04 para c1 =', round(u_04[1]*100,2),'%')

Incerteza relativa de r04 para c0 = 1.71 %
Incerteza relativa de r04 para c1 = 2.16 %


### Condição de Distribuição Normal de 150°C e fixo 60 MPa. Condição 05

In [37]:
T0 = 100+273.15 # Temperature in Kelvin
P5 = 60000    # Pressure in kPa
# This function evaluating a normal samples of bounds of P and T values
sigma = 0.01*T0 
ns = 100
mu = T0

samples_n150_60 = (sorted(np.random.normal(mu,sigma,ns).tolist()))

In [38]:
rho_n150_60 = [[] for _ in range(len(c))]

# Evaluanting for each mixture
for i in range(0,len(c)):
    for j, tempo in zip(range(len(samples_n150_60)), tqdm(range(len(samples_n150_60)))):
        rho_n150_60[i].append(GERG2008rho(c[i],P4,samples_n150_60[j]))

 99%|███████████████████████████████████████████████████████████████████████████████████████████████████████████▉ | 99/100 [19:59<00:12, 12.12s/it]


In [39]:
# Evaluating r
r_05 = []
for i in range(len(rho_standard)):
    r_05.append( rho_n150_60[i]/rho_standard[i])

# Evaluating u_r
ur_05 = []
for i in range(0,len(c)):
    ur_05.append(np.std(r_05[i]))

# Evaluating the uncertaninty portion with 95% confidence
u_05 = []
for i in range(0,len(c)):
    u_05.append(((1.96*np.std(r_05[i]))/np.mean(r_05[i])))

In [40]:
print('Incerteza relativa de r04 para c0 =', round(u_05[0]*100,2),'%')
print('Incerteza relativa de r04 para c1 =', round(u_05[1]*100,2),'%')

Incerteza relativa de r04 para c0 = 1.83 %
Incerteza relativa de r04 para c1 = 2.3 %


### Condição de Distribuição Normal de 150°C e fixo 70 MPa. Condição 06

In [41]:
T0 = 100+273.15 # Temperature in Kelvin
P6 = 60000    # Pressure in kPa
# This function evaluating a normal samples of bounds of P and T values
sigma = 0.01*T0 
ns = 100
mu = T0

samples_n150_70 = (sorted(np.random.normal(mu,sigma,ns).tolist()))

In [42]:
rho_n150_70 = [[] for _ in range(len(c))]

# Evaluanting for each mixture
for i in range(0,len(c)):
    for j, tempo in zip(range(len(samples_n150_70)), tqdm(range(len(samples_n150_70)))):
        rho_n150_70[i].append(GERG2008rho(c[i],P4,samples_n150_70[j]))

 99%|███████████████████████████████████████████████████████████████████████████████████████████████████████████▉ | 99/100 [20:07<00:12, 12.20s/it]


In [43]:
# Evaluating r
r_06 = []
for i in range(len(rho_standard)):
    r_06.append( rho_n150_70[i]/rho_standard[i])

# Evaluating u_r
ur_06 = []
for i in range(0,len(c)):
    ur_06.append(np.std(r_06[i]))

# Evaluating the uncertaninty portion with 95% confidence
u_06 = []
for i in range(0,len(c)):
    u_06.append(((1.96*np.std(r_06[i]))/np.mean(r_06[i])))

In [44]:
print('Incerteza relativa de r04 para c0 =', round(u_06[0]*100,2),'%')
print('Incerteza relativa de r04 para c1 =', round(u_06[1]*100,2),'%')

Incerteza relativa de r04 para c0 = 1.64 %
Incerteza relativa de r04 para c1 = 2.07 %


## Cálculo das médias e incertezas da massa específica

In [62]:
rho_n150_10_95 = [[],[]]

for i in range(len(rho_n150_10)):
    for j in range(len(rho_n150_10[i])):
        if rho_n150_10[i][j] > (np.mean(rho_n150_10[i])-1.96*np.std(rho_n150_10[i])) or rho_n150_10[i][j] < (np.mean(rho_n150_10[i])+1.96*np.std(rho_n150_10[i])):
            rho_n150_10_95[i].append(rho_n150_10[i][j])

rho_n150_20_95 = [[],[]]

for i in range(len(rho_n150_20)):
    for j in range(len(rho_n150_20[i])):
        if rho_n150_20[i][j] > (np.mean(rho_n150_20[i])-1.96*np.std(rho_n150_20[i])) or rho_n150_20[i][j] < (np.mean(rho_n150_20[i])+1.96*np.std(rho_n150_20[i])):
            rho_n150_20_95[i].append(rho_n150_20[i][j])

rho_n150_30_95 = [[],[]]

for i in range(len(rho_n150_30)):
    for j in range(len(rho_n150_30[i])):
        if rho_n150_30[i][j] > (np.mean(rho_n150_30[i])-1.96*np.std(rho_n150_30[i])) or rho_n150_30[i][j] < (np.mean(rho_n150_30[i])+1.96*np.std(rho_n150_30[i])):
            rho_n150_30_95[i].append(rho_n150_30[i][j])

rho_n150_40_95 = [[],[]]

for i in range(len(rho_n150_40)):
    for j in range(len(rho_n150_40[i])):
        if rho_n150_40[i][j] > (np.mean(rho_n150_40[i])-1.96*np.std(rho_n150_40[i])) or rho_n150_40[i][j] < (np.mean(rho_n150_40[i])+1.96*np.std(rho_n150_40[i])):
            rho_n150_40_95[i].append(rho_n150_40[i][j])

rho_n150_50_95 = [[],[]]

for i in range(len(rho_n150_50)):
    for j in range(len(rho_n150_50[i])):
        if rho_n150_50[i][j] > (np.mean(rho_n150_50[i])-1.96*np.std(rho_n150_50[i])) or rho_n150_50[i][j] < (np.mean(rho_n150_50[i])+1.96*np.std(rho_n150_50[i])):
            rho_n150_50_95[i].append(rho_n150_50[i][j])

rho_n150_60_95 = [[],[]]

for i in range(len(rho_n150_60)):
    for j in range(len(rho_n150_60[i])):
        if rho_n150_60[i][j] > (np.mean(rho_n150_60[i])-1.96*np.std(rho_n150_60[i])) or rho_n150_60[i][j] < (np.mean(rho_n150_60[i])+1.96*np.std(rho_n150_60[i])):
            rho_n150_60_95[i].append(rho_n150_60[i][j])

rho_n150_70_95 = [[],[]]

for i in range(len(rho_n150_70)):
    for j in range(len(rho_n150_70[i])):
        if rho_n150_70[i][j] > (np.mean(rho_n150_70[i])-1.96*np.std(rho_n150_70[i])) or rho_n150_70[i][j] < (np.mean(rho_n150_70[i])+1.96*np.std(rho_n150_70[i])):
            rho_n150_70_95[i].append(rho_n150_70[i][j])

c0 = 4.87 |||| c1 = 3.46
c0 = 9.46 |||| c1 = 6.97
c0 = 11.35 |||| c1 = 9.82
c0 = 12.48 |||| c1 = 11.96
c0 = 13.27 |||| c1 = 13.58
c0 = 13.28 |||| c1 = 13.59
c0 = 0.16 |||| c1 = 0.05
c0 = 0.19 |||| c1 = 0.11
c0 = 0.14 |||| c1 = 0.13
c0 = 0.13 |||| c1 = 0.15
c0 = 0.12 |||| c1 = 0.15
c0 = 0.12 |||| c1 = 0.16
c0 = 0.11 |||| c1 = 0.14
